In [61]:

# Dependencies

import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

from config import api_key




In [62]:
headers = {'Authorization': 'Bearer %s' % api_key}

In [63]:
url='https://api.yelp.com/v3/businesses/search'
 
# paramaters
params = {'term':'music venue','location':'Denver','radius':20000,'limit':50}

In [64]:
response=requests.get(url, params=params, headers=headers).json()
 
# proceed only if the status code is 200
#print('The status code is {}'.format(response.status_code))

In [98]:
response


{'businesses': [{'id': '0IqSfcno328kCD374Q7R7g',
   'alias': 'your-moms-house-denver',
   'name': "Your Mom's House",
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/mMUnWRrb4k1qI7RL3iBgQg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/your-moms-house-denver?adjust_creative=nC2mzMNmRx08dqWbq5LVeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nC2mzMNmRx08dqWbq5LVeQ',
   'review_count': 47,
   'categories': [{'alias': 'musicvenues', 'title': 'Music Venues'},
    {'alias': 'danceclubs', 'title': 'Dance Clubs'},
    {'alias': 'bars', 'title': 'Bars'}],
   'rating': 4.0,
   'coordinates': {'latitude': 39.7367329, 'longitude': -104.9795145},
   'transactions': [],
   'price': '$',
   'location': {'address1': '608 E 13th Ave',
    'address2': '',
    'address3': None,
    'city': 'Denver',
    'zip_code': '80203',
    'country': 'US',
    'state': 'CO',
    'display_address': ['608 E 13th Ave', 'Denver, CO 80203']},
   'phone': '+1303860451

In [100]:
params2 = {'term':'music venue','location':'Denver','radius':20000,'limit':50,'offset':50}
response2=requests.get(url, params=params2, headers=headers).json()
response2

{'businesses': [{'id': 'kcSZHlnFAGZiX5SIurlDTw',
   'alias': 'lincolns-roadhouse-denver',
   'name': "Lincoln's Roadhouse",
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/XT8BgLRIlrhiW293FJGmPQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/lincolns-roadhouse-denver?adjust_creative=nC2mzMNmRx08dqWbq5LVeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nC2mzMNmRx08dqWbq5LVeQ',
   'review_count': 120,
   'categories': [{'alias': 'bars', 'title': 'Bars'},
    {'alias': 'cajun', 'title': 'Cajun/Creole'},
    {'alias': 'musicvenues', 'title': 'Music Venues'}],
   'rating': 4.0,
   'coordinates': {'latitude': 39.694668, 'longitude': -104.980915},
   'transactions': ['pickup', 'delivery'],
   'price': '$',
   'location': {'address1': '1201 S Pearl St',
    'address2': '',
    'address3': '',
    'city': 'Denver',
    'zip_code': '80210',
    'country': 'US',
    'state': 'CO',
    'display_address': ['1201 S Pearl St', 'Denver, CO 80210']},


In [101]:
busId=[]
busName=[]
address=[]
phoneNum=[]
review_count=[]
rating=[]
priceLevel=[]
numOfCategories=[]
categories=[]
distance=[]
noPriceData=[]

for x in range(len(response['businesses'])):
    subList=[]
    for i in range(len(response['businesses'][x]['categories'])):
        subList.append(response['businesses'][x]['categories'][i]['title'])
    categories.append(subList)
    busId.append(response['businesses'][x]['id'])
    busName.append(response['businesses'][x]['name'])
    address.append(response['businesses'][x]['location']['display_address'])
    phoneNum.append(response['businesses'][x]['display_phone'])    
    numOfCategories.append(len(response['businesses'][x]['categories']))
    try:
        priceLevel.append(response['businesses'][x]['price'])    
    except:
        priceLevel.append('N/A')
        noPriceData.append(response['businesses'][x]['id'])
    try:
        review_count.append(response['businesses'][x]['review_count'])
        distance.append(response['businesses'][x]['distance'])
        rating.append(response['businesses'][x]['rating'])
    except:
        print(f"Could not access all the information for {response['businesses'][x]['name']}")
        del categories[-1]
        del busId[-1]
        del busName[-1]
        del address[-1]
        del phoneNum[-1]
        del numOfCategories[-1]
        
for x in range(len(response2['businesses'])):
    subList=[]
    for i in range(len(response2['businesses'][x]['categories'])):
        subList.append(response2['businesses'][x]['categories'][i]['title'])
    categories.append(subList)
    busId.append(response2['businesses'][x]['id'])
    busName.append(response2['businesses'][x]['name'])
    address.append(response2['businesses'][x]['location']['display_address'])
    phoneNum.append(response2['businesses'][x]['display_phone'])    
    numOfCategories.append(len(response2['businesses'][x]['categories']))
    try:
        priceLevel.append(response2['businesses'][x]['price'])    
    except:
        priceLevel.append('N/A')
        noPriceData.append(response2['businesses'][x]['id'])
    try:
        review_count.append(response2['businesses'][x]['review_count'])
        distance.append(response2['businesses'][x]['distance'])
        rating.append(response2['businesses'][x]['rating'])
    except:
        print(f"Could not access all the information for {response['businesses'][x]['name']}")
        del categories[-1]
        del busId[-1]
        del busName[-1]
        del address[-1]
        del phoneNum[-1]
        del numOfCategories[-1]
        

In [102]:
businessDataDF=pd.DataFrame({"ID":busId,"Name":busName,"Address":address,"Phone Number":phoneNum,"Review Count":review_count,
                            "Rating":rating,"Price Level":priceLevel,"Number of Categories":numOfCategories,
                            "Categories":categories,"Distance from Denver":distance})
businessDataDF

,ID,Name,Address,Phone Number,Review Count,Rating,Price Level,Number of Categories,Categories,Distance from Denver
0,0IqSfcno328kCD374Q7R7g,Your Mom's House,"[608 E 13th Ave, Denver, CO 80203]",(303) 860-4516,47,4.0,$,3,"[Music Venues, Dance Clubs, Bars]",803.903839
1,_UXWmagvPB86UA3PE2revA,Ogden Theatre,"[935 E Colfax Ave, Denver, CO 80218]",(303) 832-1874,199,3.5,$$,3,"[Music Venues, Venues & Event Spaces, Performi...",679.836663
2,oAWkAw_C5ByIZQubcVgIBw,Bluebird Theater,"[3317 E Colfax Ave, Denver, CO 80206]",(303) 377-1666,173,4.0,$$,2,"[Performing Arts, Music Venues]",1947.117349
3,8WGzLAv1Mu_R9amiMz6RuQ,The Soiled Dove Underground,"[7401 E 1st Ave, Denver, CO 80230]",(303) 366-0007,89,4.0,$$,2,"[Music Venues, Venues & Event Spaces]",6189.800684
4,m6hFStjFi3KfRXf4v777Cw,Mission Ballroom,"[4242 Wynkoop St, Denver, CO 80216]",,43,4.0,N/A,1,[Music Venues],4524.650241
...,...,...,...,...,...,...,...,...,...,...
95,nGE69W_8ihW3gngydPQFGg,Horseshoe Lounge,"[414 E 20th Ave, Denver, CO 80205]",(303) 832-1180,212,4.0,$,1,[Lounges],1657.535487
96,gGXnG-Rf5UUr6wT8krXT5w,Williams & Graham,"[3160 Tejon St, Denver, CO 80211]",(303) 997-8886,806,4.5,$$,2,"[American (Traditional), Speakeasies]",4553.231209
97,e-iHfrp40Rvn4bP5JsgM4g,Woods Boss Brewing Company,"[2210 California St, Denver, CO 80205]",(720) 642-7177,131,4.5,$$,2,"[Breweries, Beer Bar]",2086.848521
98,qFrNF8c8vUDUOPZnezQ7jQ,Denver Jazz Trio,"[Denver, CO 80231]",(720) 338-5224,1,5.0,N/A,2,"[Jazz & Blues, Musicians]",10578.856602


range(0, 2)